In [25]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr

intron_annot = pd.read_csv('/project2/yangili1/bjf79/ChromatinSplicingQTLs/data/IntronAnnotationsFromYang.tsv.gz', sep='\t')
intron_annot['junction'] = intron_annot[['chrom', 'start', 'end', 'strand']].astype(str).agg(':'.join, axis=1)

long_table = pd.read_csv(
    '../code/SplicingAnalysis/CombinedJuncTables/NMD_KD.tsv.gz', sep='\t'
)


In [26]:
# Adjustment needed to match the introns in Yang's annotation...
long_table.stop += 1

long_table = long_table.merge(intron_annot, left_on=['chrom', 'start', 'stop', 'strand'], 
                 right_on=['chrom', 'start', 'end', 'strand'])

long_table['isNMD'] = long_table.SuperAnnotation.isin(['AnnotatedJunc_UnproductiveCodingGene', 
                                                       'UnannotatedJunc_UnproductiveCodingGene'])

long_table = long_table.loc[long_table.SuperAnnotation.isin(['AnnotatedJunc_UnproductiveCodingGene', 
                                                       'UnannotatedJunc_UnproductiveCodingGene',
                                                            'AnnotatedJunc_ProductiveCodingGene', 
                                                       'UnannotatedJunc_ProductiveCodingGene'])]

In [27]:
expressed_genes = pd.read_csv('../code/ExpressionAnalysis/polyA/ExpressedGeneList.txt', sep ='\t',
                             names = ['chrom', 'start', 'end', 'gene', 'score', 'strand'])

In [28]:
long_table = long_table.loc[long_table.gene.isin(expressed_genes.gene)]

In [21]:
long_table.loc[(long_table.Dataset=='HeLa.scr') & (long_table.isNMD)].groupby('IndID').Count.sum()/long_table.loc[(long_table.Dataset=='HeLa.scr')].groupby('IndID').Count.sum()

IndID
SRR4081222    0.008618
SRR4081223    0.009047
SRR4081224    0.008920
SRR4081237    0.007828
SRR4081238    0.008336
SRR4081239    0.008731
Name: Count, dtype: float64

In [29]:
long_table.loc[(long_table.Dataset=='HeLa.dKD') & (long_table.isNMD)].groupby('IndID').Count.sum()/long_table.loc[(long_table.Dataset=='HeLa.dKD')].groupby('IndID').Count.sum()

IndID
SRR4081246    0.029074
SRR4081247    0.029175
SRR4081248    0.029247
Name: Count, dtype: float64

In [30]:
long_table.Dataset.unique()

array(['HeLa.scr', 'HeLa.SMG6.KD', 'HeLa.SMG7.KD', 'HeLa.UPF1.KD',
       'HeLa.dKD'], dtype=object)